In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("train.csv")
len_train,col_train = train_df.shape[0],train_df.shape[1]
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
train_df = train_df.dropna()
train_df.drop(['Embarked','Name','Ticket','PassengerId'],axis=1,inplace=True)
train_df = train_df.reset_index(drop=True)
y = np.array(train_df['Survived'].copy()).reshape(len(train_df['Survived']),1)
X = train_df.drop('Survived',axis=1)

In [4]:
for i in range(X.shape[0]):
    if X.at[i,"Sex"] == "male":
        X.at[i,"Sex"] = 1
    else:
        X.at[i,"Sex"] = 0

X["Cabin"] = [ord(X.at[i,"Cabin"][0])-ord("A") for i in range(X.shape[0])]
#X["Sex"] = [X.at[i,"Sex"][0] for i range(X.shape[0])]
X["Fare"] = [round(X["Fare"][i]/75,3) for i in range(X.shape[0])]
X["Age"] = [round(X["Age"][i]/35,3) for i in range(X.shape[0])]
X.describe()


,Pclass,Age,SibSp,Parch,Fare,Cabin
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,1.191257,1.019306,0.464481,0.475410,1.049077,2.491803
std,0.515187,0.446979,0.644159,0.754617,1.017965,1.889441
min,1.000000,0.026000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.686000,0.000000,0.000000,0.396000,1.000000
50%,1.000000,1.029000,0.000000,0.000000,0.760000,2.000000
75%,1.000000,1.357000,1.000000,1.000000,1.200000,3.500000
max,3.000000,2.286000,3.000000,4.000000,6.831000,19.000000


In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4)
X_train = X_train.T
y_train = y_train.T
X_test = X_test.T
y_test = y_test.T

In [6]:
X_train = np.array(X_train, dtype=float)
X_test = np.array(X_test, dtype=float)
X_train.shape

(7, 109)

In [7]:
y_test.shape

(1, 74)

In [8]:
def sigmoid(z):
    
    s = 1/(1+np.exp(-z))
  
    
    return s

def initialize_params(layer_dim): #layer_dim = [7, 4, 1]
    
    parameters = {}
    for l in range(1,len(layer_dim)):
        parameters["w"+str(l)] = np.random.randn(layer_dim[l],layer_dim[l-1])
        parameters["b"+str(l)] = np.zeros((layer_dim[l], 1))
        
    return parameters
    
def compute_loss(a3, Y):
    
    """
    Implement the loss function
    
    Arguments:
    a3 -- post-activation, output of forward propagation
    Y -- "true" labels vector, same shape as a3
    
    Returns:
    loss - value of the loss function
    """
    
    m = Y.shape[1]
    logprobs = np.multiply(-np.log(a3),Y) + np.multiply(-np.log(1 - a3), 1 - Y)
    loss = 1./m * np.nansum(logprobs)
    
    return loss

    
def fwd_prop(parameters, X):
    m = X.shape[1]
    w1 = parameters["w1"] #(4,7)
    b1 = parameters["b1"] #(4,1)
    w2 = parameters["w2"] #(1,4)
    b2 = parameters["b2"] #(1,1)
    
    z1 = np.dot(w1,X) + b1
    A1 = sigmoid(z1)
    z2 = np.dot(w2,A1) + b2
    A2 = sigmoid(z2)
    
    
    cache = (z1, A1, w1, b1, z2, A2, w2, b2)
    
    return A2, cache
    
    
    
def back_prop(X, Y, cache):
    m = X.shape[0]
    z1, A1, w1, b1, z2, A2, w2, b2 = cache
    dz2 = 1/m  * (A2 - Y)
    dw2 = np.dot(dz2,A1.T)
    db2 = np.sum(dz2, axis=1, keepdims = True)
    
    da1 = np.dot(w2.T, dz2)
    dz1 = np.multiply(da1, np.int64(A1 > 0))
    dw1 = np.dot(dz1, X.T)
    db1 = np.sum(dz1, axis=1, keepdims = True)
    
    grads = {"dw2":dw2,
             "db2":db2,
             "dw1":dw1,
             "db1":db1}
    
    return grads
    
    
#     Z = np.dot(w.T,X)+b
    
#     A = sigmoid(Z)
#     cost = (-1/m)*np.sum(np.multiply(y,np.log(A))+np.multiply((1-y),np.log(1-A)))
#     dw = (1/m)*np.dot(X,(A-y).T)
#     db = (1/m)*np.sum(A-y)
#     grads = {"dw": dw,
#              "db": db}
    
#     return grads, cost


In [9]:
def optimize(w, b, X, y, num_iter, learning_rate, print_cost=True):
    costs = []
    for i in range(num_iter):
        grads, cost = fwd_bck_prop(w,b,X,y)
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        
        if i % 1000 == 0:
            costs.append(cost)
            
        if print_cost and i%1000 == 0:
            print("for iteration {} cost : {}".format(i,cost))
        
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
        
    return params, grads, costs

In [10]:
def predict(parameters, X):
    
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    
    A3, cache = fwd_prop(parameters, X)
    Y_prediction = np.array([[1 if A3[0, i] > 0.5 else 0 for i in range(m)]])
    
    return Y_prediction
    
    
    
def update_params(parameters, grads, learning_rate):
    
    
    L = len(parameters) // 2 

    
    for k in range(L):
        parameters["w" + str(k+1)] = parameters["w" + str(k+1)] - learning_rate * grads["dw" + str(k+1)]
        parameters["b" + str(k+1)] = parameters["b" + str(k+1)] - learning_rate * grads["db" + str(k+1)]
        
    return parameters
    
    
    
        
    
    

In [15]:
def model(X_train, y_train, X_test, y_test, num_iter=20000, learning_rate = 0.0001, print_cost=True):
    
    layer_dim = [7, 4, 1]
    parameters = initialize_params(layer_dim)
    print(parameters["w1"].shape)
    
    
    costs = []
    
    for i in range(num_iter):
        
        A2, cache = fwd_prop(parameters, X_train)
        cost = compute_loss(A2, y_train)
        grads = back_prop(X_train, y_train, cache)
        #print(grads)
        
        parameters = update_params(parameters, grads, learning_rate)
        
        
        if print_cost and i % 1000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
            costs.append(cost)
        
    
    
#     parameters, grads, costs = optimize(w, b, X_train, y_train, num_iter, learning_rate, print_cost)
    
    print("params : {}".format(parameters))
    print("grads : {}".format(grads))
#     print("costs : {}".format(costs))
    

    
    y_predict_test = predict(parameters, X_test)
    y_predict_train = predict(parameters,X_train)
    
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(y_predict_train - y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_predict_test - y_test)) * 100))
    

    
    

In [29]:
model(X_train, y_train, X_test, y_test, num_iter = 10000, learning_rate = 0.0001, print_cost = True)

(4, 7)
Cost after iteration 0: 0.6379276505062431
Cost after iteration 1000: 0.6003827902429534
Cost after iteration 2000: 0.5755734787888821
Cost after iteration 3000: 0.5560217391773005
Cost after iteration 4000: 0.5405309185219878
Cost after iteration 5000: 0.5278782330285877
Cost after iteration 6000: 0.5172106987858579
Cost after iteration 7000: 0.5080283200315926
Cost after iteration 8000: 0.5000448721598383
Cost after iteration 9000: 0.4930866055906286
params : {'w2': array([[-0.55880432, -1.17074362,  1.77659112,  0.47856409]]), 'b2': array([[0.42983485]]), 'b1': array([[-0.21862001],
       [-0.49803336],
       [ 0.60865003],
       [ 0.18917851]]), 'w1': array([[-0.54715251,  0.8519485 ,  0.48648882, -0.30429673,  0.40701327,
        -1.60941089, -0.04884037],
       [-0.37943133,  0.50290542,  1.48467676,  0.77865173,  1.10975402,
        -0.5106979 , -0.53368661],
       [ 0.65589032, -2.99513534,  0.10552807,  1.45356937,  0.61549255,
         1.53444962, -1.0343298 ],
  

In [ ]:
params = initialize_params([7,4,1])